In [2]:
#!pip install ensemble_boxes

In [2]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [3]:
# ensemble csv files
#submission_files = ['submission.csv']
submission_files = ['/opt/ml/submission_files/submission (4)_won1.csv', '/opt/ml/submission_files/submission_train_all_withtta_low_thr.csv', '/opt/ml/submission_files/submission_faster_rcnn_hrnetv2p_w40_2x_coco_(autoAug)_30.csv']
submission_df = [pd.read_csv(file) for file in submission_files]

In [4]:
image_ids = submission_df[0]['image_id'].tolist()

In [5]:
annotation = '/opt/ml/input/data/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [6]:
prediction_strings = []
file_names = []
iou_thr = 0.4

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        if len(predict_list)==0 or len(predict_list)==1:
            continue
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

Warning. Removed 22 boxes with zero area!
Warning. Removed 99 boxes with zero area!
Warning. Removed 98 boxes with zero area!
Warning. Removed 99 boxes with zero area!
Warning. Removed 33 boxes with zero area!
Warning. Removed 11 boxes with zero area!
Warning. Removed 11 boxes with zero area!
Warning. Removed 88 boxes with zero area!
Warning. Removed 98 boxes with zero area!
Warning. Removed 98 boxes with zero area!
Warning. Removed 22 boxes with zero area!
Warning. Removed 97 boxes with zero area!
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Removed 88 boxes with zero area!
Warning. Removed 77 boxes with zero area!
Warning. Removed 55 boxes with zero area!
Warning. Removed 22 boxes with zero area!
Warning. Removed 22 boxes with zero area!
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Removed 98 boxes with zero area!
Warning. Removed 98 boxes with zero area!
Warning. Removed 98 boxes wi

In [8]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission_ensemble.csv')

submission.to_csv(os.path.join('/opt/ml/submission_files/wbf_submission', 'wbf_(10_12_9).csv'), index=None)

submission.head()

,PredictionString,image_id
0,0 0.005345957333333334 165.31673 76.64492 253....,batch_01_vt/0021.jpg
1,1 0.985293447971344 0.7116420865058899 244.699...,batch_01_vt/0028.jpg
2,1 0.99063617 76.68968 173.37753 371.64917 387....,batch_01_vt/0031.jpg
3,2 0.9473936557769775 297.47955322265625 157.42...,batch_01_vt/0032.jpg
4,0 0.004589149333333333 21.38771 133.05128 388....,batch_01_vt/0070.jpg
